手写数字的识别流程
1、准备数据集包含训练集和测试集
2、将数据集的图片做尺寸归一化，保持相同大小
3、定义超参数//例如定义循环的次数。
4、构建transforms，主要是对图像做变换
5、下载、加载数据集MNIST
6、构建网络模型------------重要
7、定义训练方法
8、定义测试方法
9、开始训练模型，输出预测结果
专业名词解释
    1、参数与超参数
    参数：模型f（x,$）中的$称为模型的参数，可以通过优化算法进行学习。
    超参数：用来定义模型结构或优化策略。
    2、batch_size批处理。每次处理的数据数量
    3、epoch轮次。把一个数据集，循环运行几轮。
    4、transforms变换。主要是将图片转换为tensor，旋转图片，以及正则化
    5、nomalize正则化。模型出现过拟合现象时，降低模型复杂度。
    6、卷积层。由卷积核构建，卷积核简称为卷积，也称为滤波器。卷积的大小可以在实际需要的时候自定义其长和宽（1*1,3*3，5*5）
    7、池化层。对图片进行压缩（降采样）的一种方法，如max pooling，average pooling等
    8、激活层。激活函数的作用就是，在所有隐藏层之间添加一个激活函数，这样的输出就是一个非线性函数了，因而神经网络的表达能力就更加强大了
    9、损失函数。在深度学习中，损失函数反应了模型最后预测效果和实际真值之间的差距，可以用来分析训练过程的好坏、模型是否收敛等、
    例如均方损失、交叉熵损失

In [1]:
#加载必要的库
import torch
#nn网络库
import torch.nn as nn
import torch.nn.functional as F
#优化器
import torch.optim as optim
from torchvision import datasets,transforms

In [2]:
#定义超参数
#batch_size批处理。每次处理的数据数量,
#意思为将海量的数据分批输入电脑中减少压力，一般设置为64,128，根据设备的性能调节
BATCH_SIZE = 16 
#如果有GPU则用GPU训练，否则用cpu训练
DEVICE = torch.device("cuda" if torch.cuda.is_available else "cpu")
#训练数据集的轮次，意思是你可以选择将6万张照片训练一次、10次、100次。。。。
EPOCHS = 10 

In [3]:
#构建pipeline也就transfoms,对图像进行处理
#可以对图片转换成tensor，旋转图片，以及正则化，明亮度，等等进行处理
pipeline = transforms.Compose([
    transforms.ToTensor(),#将图片转换成tensor格式
    #nomalize正则化。模型出现过拟合现象时，降低模型复杂度
    #过拟合的含义：例如你训练出的模型只认识你自己写的字迹，你朋友写的字迹就不认识了。只认识见过的一模一样的，稍微改变一点就不认识了。
    transforms.Normalize((0.1307,),(0.3081)) #正则化，如果不确定取多少值，那就选择这个官网给的值
])

In [4]:
#下载、加载数据·
#dataloader对数据进行处理
from torch.utils.data import DataLoader
#下载数据集
#train训练集train是true，test集train就要是false假了，transform=pipline是将数据transfom对数据处理
train_set = datasets.MNIST("data",train=True,download=True,transform=pipeline)
test_set =datasets.MNIST("data",train=False,download=True,transform=pipeline)
#加载数据
#shuffle=True是将图片打乱，是训练图片无顺序，有助于模型精度提高
train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
test_loader =  DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True)


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


#此处仅为学习如何打开下载的文件,  显示mnist中的图片
with open("./data/MNIST/raw/train-images-idx3-ubyte","rb")as f:
    file =f.read()
image1 =[int(str(item).encode('ascii'),16)for item in file[16:16+784]]
print(image1)
import cv2
import numpy as np
imagel_np =np.array(imagel,dtype=np.uint8).reshape(28,28,1)
print(image1_np.shape)
cv2.imwrite("digit.jpg",imagel_np)

In [5]:
#构建网络模型
class Digit(nn.Module):
    def _init_(self):
        super()._init_()
        self.conv1 = nn.Conv2d(1,10,5)#卷积层第一层 1: 灰度图片通道 10：输出通道 5：kernel卷积核
        self.conv2 = nn.Conv2d(10,20,3)#第二层 10：输入通道 20输出通道 3：kernel卷积核
        self.fc1 = nn.Linear(20*10*10,500)#20*10*10：输入通道 ，500输出通道  全连接层
        self.fc2 = nn.Linear(500,10)#500:输入通道 10输出通道
        #前向传播
    def forward(self,x):
        input_size = x.size(0)#batch_size *1*28*28为1灰度28是像素
        x = self.conv1(x)#batch*1*28*28 ，输出：batch*10*24*24   24是28-5+1=24
        x = F.relu(x)
        x = F.max_poo12d(x,2,2)#池化层
        x = self.conv2(x)
        x = F.relu(x)
        x = x.view(input_size,-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x,dim=1)
        return output

In [6]:
#定义优化器
model = Digit().to(DEVICE)
optimizer = optim.Adam(model.parameters())

ValueError: optimizer got an empty parameter list

In [ ]:
#定义训练的函数方法
def train_model(model,device,train_loader,optimizer,epoch):
    #模型训练
    model.train()
    for batch_index,(data, target) in enumerate(train_loader):
        data,target=data.to(device),target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        pred = output.max(1,keepdim=True)
        loss.backward()
        optimizer.step()
        if batch_index % 3000 ==0: 
            print("Train Epoch:{}\t Loss : {:.6f}".format(epoch,loss.item()))

In [ ]:
#定义测试方法
def test_model(model,device,test_loader):
    model.eval()
    correct =0.0
    test_loss = 0.0
    with torch.no_grad():
        for data,target in test_loader:
            data,target=data.to(device),target.to(device)
            output = model(data)
            test_loss =F.cross_entropy(outupt,target).item()
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test --Average loss:{:.4f},Accuracy : {:.3f}\n".format(
            test_loss, 100.0*correct/len(test_loader.dataset)))
            

In [ ]:
#调用方法7/8
for epoch in range(1,EPOCHS +1)：
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

